In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

# To display the all columns in pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# read the training data
df = pd.read_csv("train.csv")

In [ ]:
# Number of rows/columns
df.shape

In [ ]:
# print the data to see sample rows
df.head(5)

In [ ]:
# Get some basic stats <- not much help here as there are 50+ columns
df.describe()

In [ ]:
# Check to see missing values. As seen below, Alley doesn't have much data. FireplaceQu has 50% missing. PoolQC, Fence and Misc are also rare.
# Count doesn't include NaN values. If you want length, use len(df.index). Alsom df.dtypes gives object types
df.info()


##### Plot the histogram to see the SalePrice. This shows that most of the houses are within 100k-250k range. (almost ~1100)

In [ ]:
df.SalePrice.hist()

#### Features
There are multiple ideas that can be explored next. Since there are 81 features, I think not all are important. We can bucket these features into categories and can find correlatiom in them. Two things to keep in mind - handle NaN values and convert object to int. 

#### Basement:
BsmtQual         1423   
BsmtCond         1423   
BsmtExposure     1422   
BsmtFinType1     1423   
BsmtFinSF1       1460   
BsmtFinType2     1422   
BsmtFinSF2       1460   

#### Garage:
GarageType       1379   
GarageYrBlt      1379   
GarageFinish     1379   
GarageCars       1460   
GarageArea       1460   
GarageQual       1379   
GarageCond       1379   

So from the above two types, I we can see that for Basement, NA	No Basement is just NaN. So we need to convert NaNs to NA. Same with garage - convert NaNs to NA No Garage. The ints will still be 0. 
   
Also do the same for Fence (NaN -> NA), Pool (NaN -> NA), Misc (NaN->NA)

For  LotFrontage -> NaN should be converted to 0 as it is an int type

In [ ]:
# Filling missing values
df['BsmtQual'] = df['BsmtQual'].fillna("NA")
df['BsmtCond'] = df['BsmtCond'].fillna("NA")
df['BsmtExposure'] = df['BsmtExposure'].fillna("NA")
df['BsmtFinType1'] = df['BsmtFinType1'].fillna("NA")
df['BsmtFinType2'] = df['BsmtFinType2'].fillna("NA")

df['GarageType'] = df['GarageType'].fillna("NA")
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0.0)
df['GarageFinish'] = df['GarageFinish'].fillna("NA")
df['GarageQual'] = df['GarageQual'].fillna("NA")
df['GarageCond'] = df['GarageCond'].fillna("NA")

df['FireplaceQu'] = df['FireplaceQu'].fillna("NA")

df['LotFrontage'] = df['LotFrontage'].fillna(0.0)


##### Starting feature engineering. Taking each column and doing some analysis to figure out if we will keep the column or drop the column

In [ ]:
df1 = pd.crosstab(df['SalePrice'], df['MSSubClass'])
df1.plot(subplots=True)

In [ ]:
df.hist(column="SalePrice",by="MSSubClass",bins=10)

In [ ]:
# comparing the value counts with histogram, we can see that the top three entries have similar shape but since 
# data is sparse, we can keep this. We will keep MSSubClass
df.MSSubClass.value_counts()


In [ ]:
df1 = pd.crosstab(df['SalePrice'], df['MSZoning'])
df1.plot(subplots=True)

In [ ]:
# RL and RM looks like same plots. The rest three add up to 91. We will drop MSZoning.
print df.MSZoning.value_counts()
df.hist(column="SalePrice",by="MSZoning",bins=10)

In [ ]:
#Determine pivot table. We can see that higher the LotArea, more the price. We will keep LotArea
impute_grps = df.pivot_table(values=["SalePrice"], index=["LotArea"], aggfunc=np.mean).plot()
print impute_grps


In [ ]:
# Reg and IR1 has the same shape. So not much info here. We will drop LotShape
print df.LotShape.value_counts()
df.hist(column="SalePrice",by="LotShape",bins=10)

In [ ]:
# Street is not a good signal. We will drop Street
print df.Street.value_counts()
df.hist(column="SalePrice",by="Street",bins=10)

In [ ]:
# Land contour doesn't have a lot of variation. We will drop LandContour
print df.LandContour.value_counts()
df.hist(column="SalePrice",by="LandContour",bins=10)


In [ ]:
# Utilities is not a good column as there is no variation. We will drop Utilities
df['Utilities'].value_counts()

In [ ]:
# On the edge for this. We can keep LotConfig
print df.LotConfig.value_counts()
df.hist(column="SalePrice",by="LotConfig",bins=10)

In [ ]:
# More than 90% of the data is from a single value. We will drop LandSlope
print df.LandSlope.value_counts()
df.hist(column="SalePrice",by="LandSlope",bins=10)

In [ ]:
# We can keep this for now as there is some value here. We will keep Neighborhood
print df.Neighborhood.value_counts()
df.hist(column="SalePrice",by="Neighborhood",bins=10)

In [ ]:
# Not sure of this signal. We can revisit it later. We will keep Condition1
print df.Condition1.value_counts()
df.hist(column="SalePrice",by="Condition1",bins=10)

In [ ]:
# Most of condition 2 is normal. We will drop Condition2
print df.Condition2.value_counts()
df.hist(column="SalePrice",by="Condition2",bins=10)

In [ ]:
# On the edge. We will revisit it. We will keep BldgType
print df.BldgType.value_counts()
df.hist(column="SalePrice",by="BldgType",bins=10)

In [ ]:
# Is there a way to combine HouseStyle and BldgType? We can find some correlation? We will keep HouseStyle
print df.HouseStyle.value_counts() 
df.hist(column="SalePrice",by="HouseStyle",bins=10)

In [ ]:
# Not sure how to find correlation as both the values are objects.
df.filter(items=["BldgType", "HouseStyle"]).corr()

In [ ]:
# We will keep OverallQual
print df.OverallQual.value_counts()
df.hist(column="SalePrice",by="OverallQual",bins=10)

In [ ]:
# Is there a correlation with OverallQual? We will keep OverallCond
print df.OverallCond.value_counts()
df.hist(column="SalePrice",by="OverallCond",bins=10)

In [ ]:
# Check for correlation
df.filter(items=["OverallQual", "OverallCond"]).corr()

In [ ]:
# YearBuilt is a good signal as the price increases as teh year increases. We will keep YearBuilt
df.pivot_table(values=["SalePrice"], index=["YearBuilt"], aggfunc=np.mean).plot()
#print df.YearBuilt.value_counts()


In [ ]:
# looks like a good signal. We need to find correlation with year built.
#print df.YearRemodAdd.value_counts()
df.pivot_table(values=["SalePrice"], index=["YearRemodAdd"], aggfunc=np.mean).plot()

# It has around 0.6 correlation. We will drop YearRemodAdd
df.filter(items=["YearBuilt", "YearRemodAdd"]).corr()

In [ ]:
# Gable and Hip has similar histograms. This signal is not adding value. We will drop RoofStyle
print df.RoofStyle.value_counts()
df.hist(column="SalePrice",by="RoofStyle",bins=10)

In [ ]:
# We will drop RoofMatl
print df.RoofMatl.value_counts()
df.hist(column="SalePrice",by="RoofMatl",bins=10)

In [ ]:
# Clean up the values
df['Exterior2nd'] = df['Exterior2nd'].str.replace("CmentBd","CemntBd")
df['Exterior2nd'] = df['Exterior2nd'].str.replace("Wd Shng","WdShing")

# only 139 values are different. Hence both of them are highly correlated. Hence just need once. We will keep Exterior1st
# We will drop Exterior2nd
df[df.Exterior1st != df.Exterior2nd].filter(items=["Exterior1st", "Exterior2nd", "SalePrice"]).count()


In [ ]:
# Not sure about this as 50% of values are None (which is a valid type). We will keep MasVnrType
print df.MasVnrType.value_counts()
df.hist(column="SalePrice",by="MasVnrType",bins=10)

In [ ]:
# MasVnrArea is a good signal to keep. We will keep MasVnrArea
#print df.MasVnrArea.value_counts()
df.pivot_table(values=["SalePrice"], index=["MasVnrArea"], aggfunc=np.mean).plot()


In [ ]:
# It can be a okay signal. We will keep ExterQual
print df.ExterQual.value_counts()
print df.filter(items=['ExterQual', 'ExterCond']).apply(pd.Series.value_counts)

df.hist(column="SalePrice",by="ExterQual",bins=10)

In [ ]:
# Doesn't look like a good signal as TA+Gd covers most data. We will drop ExterCond
print df.ExterCond.value_counts()
df.hist(column="SalePrice",by="ExterCond",bins=10)

In [ ]:
# Need to revisit. We will keep Foundation
print df.Foundation.value_counts()
df.hist(column="SalePrice",by="Foundation",bins=10)

In [ ]:
# Basement analysis
# BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF

# BsmtQual is a good signal as seen in the histograms. We will keep BsmtQual
print df.BsmtQual.value_counts()
df.hist(column="SalePrice",by="BsmtQual",bins=10)

# Gd and TA are almost same. Fa + Po is just 47/1460. We will drop BsmtCond
print df.BsmtCond.value_counts()
df.hist(column="SalePrice",by="BsmtCond",bins=10)

# The histograms for top values look the same. Revisit this later. We will drop BsmtExposure
print df.BsmtExposure.value_counts()
df.hist(column="SalePrice",by="BsmtExposure",bins=10)


In [ ]:
# As seen below: TotalBsmtSF = BsmtFinSF1 + BsmtFinSF2 + BsmtUnfSF
df.filter(items=["BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "SalePrice"]).head(10)

In [ ]:
# Based on histograms and above table, this can be an useful signal. We will keep BsmtFinType1
print df.BsmtFinType1.value_counts()
df.hist(column="SalePrice",by="BsmtFinType1",bins=10)

# 1256/1460 is Unf. Not a lot of variation in other items. We will drop BsmtFinType2
print df.BsmtFinType2.value_counts()
df.hist(column="SalePrice",by="BsmtFinType2",bins=10)

# Based on above table, this can be an useful signal. We will keep BsmtFinSF1
df.pivot_table(values=["SalePrice"], index=["BsmtFinSF1"], aggfunc=np.mean).plot()

# BsmtFinSF2 will be mostly 0 as there is a different column for Unf. We will drop BsmtFinSF2
df.pivot_table(values=["SalePrice"], index=["BsmtFinSF2"], aggfunc=np.mean).plot()

# BsmtUnfSF will be populated whenever there is unf in basement. Hence we can keep this over TotalBsmtSF 
# We will keep BsmtUnfSF
df.pivot_table(values=["SalePrice"], index=["BsmtUnfSF"], aggfunc=np.mean).plot()

# TotalBsmtSF = BsmtFinSF1 + BsmtFinSF2 + BsmtUnfSF. Since BsmtFinSF2 will be zero for 91.4% of times, BsmtFinSF1 + BsmtUnfSF 
# will a good indication for TotalBsmtSF. We will drop TotalBsmtSF
df.pivot_table(values=["SalePrice"], index=["TotalBsmtSF"], aggfunc=np.mean).plot()

# The graphs for BsmtFinSF1 and TotalBsmtSF are similar. The correlation is 0.52 and hence we can drop TotalBsmtSF
print df.filter(items=["BsmtFinSF1", "TotalBsmtSF"]).corr()

# There is a -0.49 correlation but I still feel that we can keep both. We can revisit it later to change it
print df.filter(items=["BsmtFinSF1", "BsmtUnfSF"]).corr()

### Note: One thing I need to understand is how can we tell about these dependent cols to the ML algo. E.g. BsmtFinType1 
### and BsmtFinSF1 are related but ML algo will think about them as independent cols. We lose informtion here. 


In [ ]:
# Grav + Wall = 11/1460. For these two, the Saleprice is < 120k. But the amount is less to make any impact. 
# We will drop Heating
print df.Heating.value_counts()
df.hist(column="SalePrice",by="Heating",bins=10)

In [ ]:
# This can be a good signal as most house prices are < 200k for TA + Gd but there are around 200 houses whose saleprice
# is >200k as it is Ex. We will keep HeatingQC
print df.HeatingQC.value_counts()
df.hist(column="SalePrice",by="HeatingQC",bins=10)

In [ ]:
# Need to revisit it. If the value is N, then we can see that the house price will be less than 250k. 
# We will keep CentralAir 
print df.CentralAir.value_counts()
print df[df.SalePrice < 70000].CentralAir.value_counts()
df.hist(column="SalePrice",by="CentralAir",bins=10)

In [ ]:
# Need to revisit it.Not enough data. We will drop Electrical
print df.Electrical.value_counts()
df.hist(column="SalePrice",by="Electrical",bins=10)

In [ ]:
# Bedroom/Bathroom sizes
# 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, TotRmsAbvGrd

df1 = df.filter(items=["1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea", "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "TotRmsAbvGrd", "SalePrice"])

# Just 26 values for LowQualFinSF. We will drop LowQualFinSF
print df1[df1.LowQualFinSF != 0].LowQualFinSF.count()

# GrLivArea = 1stFlrSF + 2ndFlrSF. We will drop 1stFlrSF and 2ndFlrSF. We will keep GrLivArea

# Need to revisit if we want BedroomAbvGr/TotRmsAbvGrd as it directly correlates to GrLivArea. It is 0.82. 
# We will drop TotRmsAbvGrd and BedroomAbvGr
print df.BedroomAbvGr.value_counts()
print df.TotRmsAbvGrd.value_counts()
df.pivot_table(values=["SalePrice"], index=["BedroomAbvGr"], aggfunc=np.mean).plot()
df.pivot_table(values=["SalePrice"], index=["TotRmsAbvGrd"], aggfunc=np.mean).plot()
df.pivot_table(values=["SalePrice"], index=["GrLivArea"], aggfunc=np.mean).plot()
df.filter(items=["BedroomAbvGr", "TotRmsAbvGrd", "GrLivArea" ]).corr()

In [ ]:
# We will drop BsmtFullBath, BsmtHalfBath, FullBath, HalfBath. We will keep df1['BsmtBath'] = df1['BsmtFullBath'] + df1['BsmtHalfBath']
# and df1['Bath'] = df1['FullBath'] + df1['HalfBath']
df1 = df.filter(items=["BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "SalePrice"])
df1['BsmtBath'] = df1['BsmtFullBath'] + df1['BsmtHalfBath']
df1['Bath'] = df1['FullBath'] + df1['HalfBath']

print df1.head(10)
#print df.BsmtFullBath.value_counts()
#print df.BsmtHalfBath.value_counts()
#print df.FullBath.value_counts()
#print df.HalfBath.value_counts()

df1.hist(column="SalePrice",by="BsmtBath",bins=10)
df1.hist(column="SalePrice",by="Bath",bins=10)
#df.hist(column="SalePrice",by="FullBath",bins=10)
#df.hist(column="SalePrice",by="HalfBath",bins=10)

# No strong correlation
df1.filter(items=["BsmtBath", "Bath"]).corr()

In [ ]:
# Doesn't look like a good signal as have 1 or 2 kitchen don't change much. We will drop KitchenAbvGr
print df.KitchenAbvGr.value_counts()
df.hist(column="SalePrice",by="KitchenAbvGr",bins=10)

In [ ]:
# Based on histograms, this looks like a good signal. We will keep KitchenQual
print df.KitchenQual.value_counts()
df.hist(column="SalePrice",by="KitchenQual",bins=10)

In [ ]:
# Need to revisit it. I think we can club them together to make a binary - Typ / No Typ. 
# Less than 7% of the houses have any deductions. We will drop Functional
print df.Functional.value_counts()
df.hist(column="SalePrice",by="Functional",bins=10)

In [ ]:
# Fireplaces is a good signal as having a fireplace incresaes the SalePrice. We will keep Fireplaces
print df.Fireplaces.value_counts()
df.hist(column="SalePrice",by="Fireplaces",bins=10)

In [ ]:
# FireplaceQu depends on number of Fireplaces. There are almost 50% houses without fireplaces. 
# Need to find a way to combine this with Fireplaces. I can see this as a useful signal as 
# Salesprice(Gd) > Salesprice(TA) > Salesprice(Fa). So need to revisit it. We will drop FireplaceQu
print df.FireplaceQu.value_counts()
df.hist(column="SalePrice",by="FireplaceQu",bins=10)

In [ ]:
# Garage
# GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond
df.filter(items=["GarageType", "GarageYrBlt", "GarageFinish", "GarageCars", "GarageArea", "GarageQual", "GarageCond", "SalePrice"]).head(10)

In [ ]:
# Removing the entries that doesn't have a garage, we can see 0.82 correlation between YearBuilt and GarageYrBlt. 
# We will drop GarageYrBlt
df[df.GarageYrBlt != 0].filter(items=["YearBuilt", "YearRemodAdd", "GarageYrBlt"]).corr()

In [ ]:
# Most are TA + NA. Not much useful info. We will drop GarageQual
print df.GarageQual.value_counts()
df.hist(column="SalePrice",by="GarageQual",bins=10)

# Most are TA + NA. Not much useful info. We will drop GarageCond
print df.GarageCond.value_counts()
df.hist(column="SalePrice",by="GarageCond",bins=10)

# Looks like a good signal. We will keep GarageFinish
print df.GarageFinish.value_counts()
df.hist(column="SalePrice",by="GarageFinish",bins=10)

# We can keep this as the price for NA is less then Attach/Detach. We will keep GarageType
print df.GarageType.value_counts()
df.hist(column="SalePrice",by="GarageType",bins=10)


In [ ]:

### Is there a correlation with GarageType and GarageFinish? Data shows that Attchd is mostly Rfn and Detchd is Unf.
df1 = pd.crosstab(df['GarageFinish'], df['GarageType'])
df1

# Looking at the table, Unf is both for Attchd and Detchd. So we can keep this data.

In [ ]:
# GarageCars and GarageArea are related. We can revisit to see which one to keep
# We will keep GarageCars. We will drop GarageArea
print df.GarageCars.value_counts() 
df.pivot_table(values=["SalePrice"], index=["GarageCars"], aggfunc=np.mean).plot()

df.pivot_table(values=["SalePrice"], index=["GarageArea"], aggfunc=np.mean).plot()

# Having more cars means bigger GarageArea
df.hist(column="GarageArea",by="GarageCars",bins=10)


In [ ]:
# N/P has similar outlines. Data is sparse. We will drop PavedDrive
print df.PavedDrive.value_counts()
df.hist(column="SalePrice",by="PavedDrive",bins=10)

In [ ]:
# Porch
# Porch Analysis - Based on the graphs below, OpenPorchSF is a very ambigious value as it doesn't so any incresae or 
# decrease in SalePrice with increase in SF value. The rest three are good samples. Instead of three individual columns,
# we can aggregate it into a single column. We will drop OpenPorchSF. We will keep df['AggPorch'] = df["ScreenPorch"] + df["3SsnPorch"] + df["EnclosedPorch"]

# Now lets see porch data
df1 = df.filter(items=['ScreenPorch', '3SsnPorch', 'EnclosedPorch', 'OpenPorchSF', 'SalePrice'])
df1['AggPorch'] = df1["ScreenPorch"] + df1["3SsnPorch"] + df1["EnclosedPorch"] #+ df1["OpenPorchSF"]
print df1.head(10)
print df1.describe()

# Check price variation with porch
df1.pivot_table(values=["SalePrice"], index=["ScreenPorch"], aggfunc=np.mean).plot()
df1.pivot_table(values=["SalePrice"], index=["3SsnPorch"], aggfunc=np.mean).plot()
df1.pivot_table(values=["SalePrice"], index=["EnclosedPorch"], aggfunc=np.mean).plot()
df1.pivot_table(values=["SalePrice"], index=["OpenPorchSF"], aggfunc=np.mean).plot()
df1.pivot_table(values=["SalePrice"], index=["AggPorch"], aggfunc=np.mean).plot()

porchTot = []
# only 116/1460 
print "ScreenPorch: " + str(len(df[df['ScreenPorch'] != 0]))

# only 24/1460
print "3SsnPorch: " + str(len(df[df['3SsnPorch'] != 0]))

# only 208/1460
print "EnclosedPorch: " + str(len(df[df['EnclosedPorch'] != 0]))

# only 804/1460
print "OpenPorchSF: " + str(len(df[df['OpenPorchSF'] != 0]))

# So 1152/1460 houses had a some kind of porch
sumPorch = len(df[df['ScreenPorch'] != 0]) + len(df[df['3SsnPorch'] != 0]) + len(df[df['EnclosedPorch'] != 0]) + len(df[df['OpenPorchSF'] != 0])
print "sumPorch: " + str(sumPorch)



In [ ]:
# Only less than 50% of the houses has WoodDeckSF. Even those house that has WoodDeckSF, we can't see a clear trend.
# We will drop WoodDeckSF
print df[df.WoodDeckSF > 0].WoodDeckSF.count()

df[df.WoodDeckSF > 0].pivot_table(values=["SalePrice"], index=["WoodDeckSF"], aggfunc=np.mean).plot()

df[df.WoodDeckSF == 0].hist(column="SalePrice",by="WoodDeckSF",bins=10)

In [ ]:
# Only seven rows i.e. houses had a pool. We will drop PoolArea and PoolQC. 
print len(df[df.PoolArea != 0])
df[df.PoolArea > 0].filter(items=['PoolArea', 'SalePrice'])

In [ ]:
# only 54 misc features are in the dataset. So 54/1460  = ~3.7%. Also, the shed is 49/54 misc vals and it
# doesn't provide any info. We will drop MiscVal and MiscFeature
print df.MiscFeature.count()
print df.MiscFeature.value_counts()
print df[df.MiscVal > 0].filter(items=['MiscFeature', 'SalePrice']).head(5)

df[df.MiscFeature.notnull()].hist(column="SalePrice",by="MiscFeature",bins=10)

In [ ]:
# More houses are sold in summer months. Houses in June are bit costlier. We can revisit this later to do another analysis
# We will drop MoSold
print df.MoSold.value_counts()
df.hist(column="SalePrice",by="MoSold",bins=10)

In [ ]:
# The mean prices of houses sold in 2008 fell by 10k from 2007 (financial crisis). People didn't buy anything costlier 
# than 450k in 2008. Other 4 years, there was atleast one house ~600k. Nothing glaring come out of this. 
# We can keep YrSold
print df.YrSold.value_counts()
df.pivot_table(values=["SalePrice"], index=["YrSold"], aggfunc=np.mean).plot()
df.hist(column="SalePrice",by="YrSold",bins=10)

In [ ]:
# The min price of New type was 113k and the mean price is 10k higher than WD type. Other types are sparse
# We can revisit it later. We will keep SaleType
print df.SaleType.value_counts()
df.hist(column="SalePrice",by="SaleType",bins=10)
print df[df.SaleType=="New"].SalePrice.describe()
print df[df.SaleType=="WD"].SalePrice.describe()

In [ ]:
# Normal, partial and Abnorml kind of look the same. We will drop SaleCondition
print df.SaleCondition.value_counts()
df.hist(column="SalePrice",by="SaleCondition",bins=10)

##### Features To keep (Total: 32 columns):
MSSubClass   
LotArea   
LotConfig   
Neighborhood   
Condition1   
BldgType   
HouseStyle   
OverallQual   
OverallCond   
YearBuilt   
Exterior1st   
MasVnrType   
MasVnrArea   
ExterQual   
Foundation   
BsmtQual   
BsmtFinType1   
BsmtFinSF1   
BsmtUnfSF   
HeatingQC   
CentralAir   
GrLivArea   
df1['BsmtBath'] = df1['BsmtFullBath'] + df1['BsmtHalfBath']   
df1['Bath'] = df1['FullBath'] + df1['HalfBath']   
KitchenQual   
Fireplaces   
GarageFinish   
GarageType   
GarageCars   
df1['AggPorch'] = df1["ScreenPorch"] + df1["3SsnPorch"] + df1["EnclosedPorch”]   
YrSold   
SaleType   


##### Features To drop (57 columns):
MSZoning   
LotShape   
Street   
LandContour   
Utilities   
LandSlope   
Condition2   
YearRemodAdd   
RoofStyle   
RoofMatl   
Exterior2nd   
ExterCond   
BsmtCond   
BsmtExposure   
BsmtFinType2   
BsmtFinSF2   
TotalBsmtSF   
Heating   
Electrical   
LowQualFinSF   
1stFlrSF   
2ndFlrSF   
TotRmsAbvGrd   
BedroomAbvGr   
BsmtFullBath   
BsmtHalfBath   
FullBath   
HalfBath   
KitchenAbvGr   
Functional   
FireplaceQu   
GarageYrBlt   
GarageQual   
GarageCond   
GarageArea   
PavedDrive   
OpenPorchSF   
ScreenPorch   
3SsnPorch   
EnclosedPorch   
WoodDeckSF   
PoolArea   
PoolQC   
MiscVal   
MiscFeature   
MoSold   
SaleCondition   

In [ ]:
# Create a final df based on the cols we need. Also, add the new three cols
finaldf = df.filter[items=["", ""]]

# check the dtypes of the df and convert objects into int by mapping them
finaldf.dtypes